In [0]:
MY_OPENAPI_KEY = '4db7f4f08cb141669f15d67fd73bfb38'                 
os.environ["OPENAI_API_KEY"] = MY_OPENAPI_KEY

In [0]:
%pip install -U openai==0.28

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Not uninstalling openai at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-459700e8-a199-488b-9fa8-312445c3fbef
    Can't uninstall 'openai'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import pandas as pd

In [0]:
### Ingest Item Data
items = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/PIMMART_UPDATED.zip')

In [0]:
#SUBSET TO COMMODITY OF INTEREST
com_name = 'COLOR COSMETICS'
items = items[items.COM_DSC == com_name]

In [0]:
# Provide one-shot examples
oneshot_dsc = 'Almay 503 Black Brown Multi-Benefit Mascara'
oneshot_output = "{'Color': 'Black;Brown', 'Confidence': 100}"

In [0]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai
def extract_attribute(input, attribute = 'color', oneshot_dsc = oneshot_dsc, oneshot_output = oneshot_output):
  openai.api_type = "azure"
  openai.api_base = "https://eastus2.api.cognitive.microsoft.com/"
  openai.api_version = "2023-07-01-preview"
  openai.api_key = os.getenv("OPENAI_API_KEY")
  #openai.api_key_path = <API-KEY>

  message_text = [{"role":"system","content":"You are an AI assistant that extracts " + attribute + " names from" + com_name + "descriptions and outputs the results (multiple values semi-colon separated) and the percent confidence score in JSON format"},{"role":"user","content":"What is the " + attribute +  " name in'" + oneshot_dsc + "?"},{"role":"assistant","content":oneshot_output},{"role":"user","content":"What is the " + attribute + " name in" + input +"?"}]

  completion = openai.ChatCompletion.create(
    engine="gpt35modelfn",
    messages = message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )
  return completion['choices'][0]['message']['content']

In [0]:
### Test output
sample_input = 'PURPLE GOLD MASCARA'
sample_output = extract_attribute(sample_input)
print(sample_input, sample_output)


PURPLE GOLD MASCARA {'Color': 'Purple;Gold', 'Confidence': 100}


In [0]:
###Generate test outputs
i = 8
sample_input = items.VND_ECOM_DSC.iloc[i]
sample_output = extract_attribute(sample_input)
print(sample_input, sample_output)

---------------------------------------------------------------------------
RateLimitError                            Traceback (most recent call last)
File <command-2875396005879710>, line 4
      2 i = 8
      3 sample_input = items.VND_ECOM_DSC.iloc[i]
----> 4 sample_output = extract_attribute(sample_input)
      5 print(sample_input, sample_output)

File <command-2875396005879708>, line 14, in extract_attribute(input, attribute, oneshot_dsc, oneshot_output)
     10 #openai.api_key_path = <API-KEY>
     12 message_text = [{"role":"system","content":"You are an AI assistant that extracts " + attribute + " names from" + com_name + "descriptions and outputs the results (multiple values semi-colon separated) and the percent confidence score in JSON format"},{"role":"user","content":"What is the " + attribute +  " name in'" + oneshot_dsc + "?"},{"role":"assistant","content":oneshot_output},{"role":"user","content":"What is the " + attribute + " name in" + input +"?"}]
---> 14 completion 